In [ ]:
from typing import TypedDict, Dict
from langgraph.graph import StateGraph, START,END
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env file

: 

In [ ]:
print("hello")

In [ ]:
llm = ChatGroq(
    temperature = 0,
    model_name = "llama-3.3-70b-versatile"
)


In [ ]:
llm.invoke("What is LangGraph?")

AIMessage(content="LangGraph is an open-source, multilingual, large language model developed by the LangGraph team. It's designed to process and generate human-like language, similar to other popular language models like LLaMA or BERT.\n\nLangGraph is unique in that it's specifically designed to be highly customizable and adaptable to various languages and tasks. The model is trained on a massive dataset of text from the internet, books, and other sources, which allows it to learn patterns and relationships in language.\n\nSome of the key features of LangGraph include:\n\n1. **Multilingual support**: LangGraph is trained on text data in multiple languages, making it a great option for applications that require language support beyond English.\n2. **Customizability**: The model can be fine-tuned for specific tasks or domains, allowing developers to adapt it to their particular use case.\n3. **Large-scale training**: LangGraph is trained on a massive dataset, which enables it to learn complex patterns and relationships in language.\n4. **Open-source**: The model is open-source, which means that developers can access and modify the code to suit their needs.\n\nLangGraph has a wide range of potential applications, including:\n\n1. **Language translation**: LangGraph can be used to improve machine translation systems, especially for low-resource languages.\n2. **Text generation**: The model can be used to generate human-like text, such as articles, stories, or chatbot responses.\n3. **Sentiment analysis**: LangGraph can be fine-tuned for sentiment analysis tasks, such as determining the emotional tone of text.\n4. **Question answering**: The model can be used to answer questions based on the text it has been trained on.\n\nOverall, LangGraph is a powerful and flexible language model that has the potential to revolutionize the way we interact with language.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 367, 'prompt_tokens': 40, 'total_tokens': 407, 'completion_time': 0.926001859, 'completion_tokens_details': None, 'prompt_time': 0.003211938, 'prompt_tokens_details': None, 'queue_time': 0.080323806, 'total_time': 0.929213797}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_93b5f9e564', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019afc04-b04e-7993-9a8a-398e9f594237-0', usage_metadata={'input_tokens': 40, 'output_tokens': 367, 'total_tokens': 407})

In [ ]:
class State(TypedDict):
    query : str
    category : str
    sentiment : str
    response : str

In [ ]:
def categorize(state: State) -> State:
    "Technical , Billing , General"
    prompt = ChatPromptTemplate.from_template(
        " Categorize the following customer query into one of these categories:" 
        " Technical, Billing, General.\n\nQuery: {query}"
    )

    chain = prompt | llm
    category = chain.invoke({"query": state["query"]}).content
    print("category result:", category)
    return {**state, "category": category}

def analyze_sentiment(state: State) -> State:
    prompt = ChatPromptTemplate.from_template(
        " Analyze the sentiment of the following customer query"
        "Response with either 'Positive', 'Negative', or 'Neutral'.\n\nQuery: {query}"
    )

    chain = prompt | llm
    sentiment = chain.invoke({"query": state["query"]}).content
    print("sentiment analysis result:", sentiment)
    return {**state, "sentiment": sentiment}


def handle_technical(state: State) -> State:
    prompt = ChatPromptTemplate.from_template(
        " Provide a technical support response for the following query:\n\nQuery: {query}"
    )

    chain = prompt | llm
    response = chain.invoke({"query": state["query"]}).content
    print("response from technical handler:", response)
    return {**state, "response": response}


def handle_billing(state: State) -> State:
    prompt = ChatPromptTemplate.from_template(
        " Provide a billing support response for the following query:\n\nQuery: {query}"
    )

    chain = prompt | llm
    response = chain.invoke({"query": state["query"]}).content
    print("response from billing handler:", response)
    return {**state, "response": response}

def handle_general(state: State) -> State:
    prompt = ChatPromptTemplate.from_template(
        " Provide a general support response for the following query:\n\nQuery: {query}"
    )

    chain = prompt | llm
    response = chain.invoke({"query": state["query"]}).content
    print("response from general handler:", response)
    return {**state, "response": response}


def escalate(state: State) -> State:
    print("escalating query due to negative sentiment.")
    return {**state, "response": "Your query has been escalated to a human agent beacause of its Negative sentiment."}


def route_query(state: State) -> State:
    if state["sentiment"] == "Negative":
        return "escalate"
    if state["category"] == "Technical":
        return "handle_technical"
    if state["category"] == "Billing":
        return "handle_billing"
    
    return "handle_general"

In [ ]:

graph = StateGraph(State)
graph.add_node("categorize",categorize)
graph.add_node("analyze_sentiment",analyze_sentiment)
graph.add_node("handle_technical",handle_technical)
graph.add_node("handle_billing",handle_billing)
graph.add_node("handle_general",handle_general)
graph.add_node("escalate",escalate)

graph.add_edge(START,"categorize")
graph.add_edge("categorize","analyze_sentiment")
graph.add_conditional_edges("analyze_sentiment",
                            route_query,
                            {
                                "handle_technical": "handle_technical",
                                "handle_billing": "handle_billing",
                                "handle_general": "handle_general", # "name" : "node_name"
                                "escalate": "escalate" 
                            }
                           )
graph.add_edge("handle_technical",END)
graph.add_edge("handle_billing",END)
graph.add_edge("handle_general",END)
graph.add_edge("escalate",END)

In [ ]:
app = graph.compile()

In [ ]:
from IPython.display import display, Image

In [ ]:
display(
    Image(
        app.get_graph().draw_mermaid_png(
        )
    )
)

In [ ]:
def run_customer_support_agent(query: str) -> Dict[str, str]:
    initial_state: State = {
        "query": query,
        "category": "",
        "sentiment": "",
        "response": ""
    }
    final_state = app.invoke(initial_state)
    return {
        "Category": final_state["category"],
        "Sentiment": final_state["sentiment"],
        "Response": final_state["response"]
    }

In [ ]:
result = run_customer_support_agent("I am unable to access my account after the recent update. Can you help?")

In [ ]:
def gradio_interface(query: str):
    result =  run_customer_support_agent(query)
    return (
        f"**Category:** {result['Category']}\n\n"
        f"**Sentiment:** {result['Sentiment']}\n\n"
        f"**Response:** {result['Response']}\n\n"
    )


# Build The Gradio App
gui = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(lines=2, label="Enter your query here"),
    outputs=gr.Markdown(label="Support Agent Response"),
    title="Customer Support Agent with LLMs",
    description="Enter a customer query to receive a categorized response along with sentiment analysis."
)

In [ ]:
if __name__ == "__main__":
    gui.launch()